In [2]:
from functions import getJSONfromURL, getPlayerStats, playerHasCollectible, getCharacterList,getPlayersPastRaids
from database import db_connect
from functions import getManifestJson
from concurrent.futures import ThreadPoolExecutor, as_completed
import matplotlib.pyplot as plt
import pandas as pd
destinyID = 4611686018468695677

In [3]:
conn = db_connect()
c = conn.cursor()

In [22]:
c.execute('''
CREATE TABLE markovpairs(
    word1 text,
    word2 text
)
''')

general = 670400011519000616
media = 670400027155365929
spoilerchat = 670402166103474190
offtopic = 670362162660900895

In [20]:
c.execute('''
CREATE TABLE messagedb(
    msg text,
    userid int,
    channelid int,
    msgid int,
    msgdate date
)
''')

In [29]:
c.execute('''
SELECT * FROM messagedb
''')
df = pd.DataFrame(c.fetchall())
df.columns = [c[0] for c in c.description]
df

,msg,userid,channelid,msgid,msgdate
0,"never tried them, although glue definitely does",264456189905993728,670400011519000616,672516124377612361,2020-01-30
1,so uhh if you shift tab in game and look at th...,220188263665762304,670400011519000616,672515643387412497,2020-01-30
2,just do it like in the *gate crash* sundial en...,153603850215882752,670400011519000616,672508494259421235,2020-01-30
3,"ah, alright",264456189905993728,670400011519000616,672508365834027019,2020-01-30
4,you just switch instantly,153159435466833921,670400011519000616,672508298926227465,2020-01-30
...,...,...,...,...,...
7020,Doing that friday <:peepoPog:545799983455272964>,367385031569702912,670400027155365929,672556613810520064,2020-01-30
7021,we are rivensbanes,377802312879570944,670400027155365929,672556626959400970,2020-01-30
7022,or are we <:thonk:670369127701020672>,377802312879570944,670400027155365929,672556655610691584,2020-01-30
7023,is what I usually use,171650677607497730,670400027155365929,672556775312064543,2020-01-30


In [7]:
c.execute(r'''

SELECT channelid FROM messagedb
GROUP BY channelid
HAVING count(msg) < 200
''')
text = list(c.fetchall())
text

[(669293365900214298, 1),
 (669655519190843412, 1),
 (669657816557944832, 6),
 (670362162660900895, 707),
 (670400011519000616, 3710),
 (670400027155365929, 1649),
 (670400057631309869, 2),
 (670401806345699369, 69),
 (670401854496309268, 204),
 (670402129340661772, 1),
 (670402166103474190, 15),
 (670637036641845258, 2),
 (670984994046345236, 1),
 (671264040974024705, 7),
 (671358241971634176, 2),
 (672541982157045791, 427)]

In [16]:
c.execute('''
SELECT msg FROM messagedb
''')
text = list(c.fetchall())
for dbquery in text:
    sentence = dbquery[0]
    words = sentence.split(' ')
    sentenceedges = zip(['__start__'] + words, words + ['__end__'])
    for (a,b) in sentenceedges:
            conn.execute(f'''
               INSERT INTO markovpairs 
               (word1, word2) 
               VALUES 
               (?,?)
            ''',(a,b))
conn.commit()

In [6]:
c.execute(f'''
               DELETE FROM markovpairs
            ''')
mp = pd.DataFrame(c.fetchall())
mp

""


In [5]:
c.execute(r'''
DELETE FROM messagedb
WHERE channelid = 
''')
conn.commit()

In [2]:
conn = db_connect()
c = conn.cursor()
c.execute('''DELETE FROM markovpairs''')#TRUNCATE
conn.commit()
c.execute('''
SELECT msg FROM messagedb
''')
text = list(c.fetchall())
for dbquery in text:
    sentence = dbquery[0]
    words = sentence.split(' ')
    sentenceedges = zip(['__start__'] + words, words + ['__end__'])
    for (a,b) in sentenceedges:
            conn.execute(f'''
            INSERT INTO markovpairs 
            (word1, word2) 
            VALUES 
            (?,?)
            ''',(a,b))
conn.commit()

In [22]:
conn.commit()
conn.close()